In [2]:
import os
import random
import numpy as np
from io import BytesIO

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from ipywidgets import widgets


## Importing data and Feature Engineering

In [1]:
# Create the .kaggle directory
!mkdir -p ~/.kaggle

# create kaggle Move the kaggle.json file (assumes it is in the current working directory)
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

!kaggle competitions download -c dogs-vs-cats



ref                                                          title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
jayaantanaath/student-habits-vs-academic-performance         Student Habits vs Academic Performance                   19512  2025-04-12 10:49:08.663000          18770        321  1.0              
adilshamim8/cost-of-international-education                  Cost of International Education                          18950  2025-05-07 15:41:53.213000           2337         39  1.0              
fatemehmohammadinia/heart-attack-dataset-tarik-a-rashid      Heart Attack Dataset                                     16250  2025-04-30 21:58:22.740000           3375         59  1.0              
glowstudygram/s

In [3]:
import zipfile

local_zip = '/content/dogs-vs-cats.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/cat_vs_dog')
zip_ref.close()

In [4]:
base_dir = '/content/cat_vs_dog'
train_zip_path = os.path.join(base_dir, 'train.zip')
test_zip_path = os.path.join(base_dir, 'test1.zip')
train_extract_path = os.path.join(base_dir, 'train')
test_extract_path = os.path.join(base_dir, 'test')


# Extract training data
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(train_extract_path)

# Extract test data
with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(test_extract_path)


In [5]:
# Check number of files in train and test directories
train_extract_path = os.path.join(train_extract_path, 'train')
test_extract_path = os.path.join(test_extract_path, 'test1')

print("Number of training images:", len(os.listdir(train_extract_path)))
print("Number of test images:", len(os.listdir(test_extract_path)))


Number of training images: 25000
Number of test images: 12500


In [6]:
# Create new directories for cats and dogs
train_cats_dir = os.path.join(train_extract_path, 'cats')
train_dogs_dir = os.path.join(train_extract_path, 'dogs')

os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)



In [7]:
import shutil

In [8]:
base_train_dir = '/content/cat_vs_dog/train/train'
# Iterate through the files in the main train directory

for filename in os.listdir(base_train_dir):
    file_path = os.path.join(base_train_dir, filename)

    # Only consider files (not directories)
    if os.path.isfile(file_path):
        # Move to 'cats' folder
        if filename.startswith('cat'):
            shutil.move(file_path, os.path.join(train_cats_dir, filename))

        # Move to 'dogs' folder
        elif filename.startswith('dog'):
            shutil.move(file_path, os.path.join(train_dogs_dir, filename))

print("Files moved successfully!")

Files moved successfully!


In [ ]:
## Validation folder creation
validation_dir = os.path.join(base_dir, 'validation')